In [1]:
# Import the dependencies
import pandas as pd
import numpy as np
from citipy import citipy
import requests
import json

# Import the API key
from config import weather_api_key

In [2]:
# Create a set of 2000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [3]:
# Get the nearest city using the citipy module

# Create a list for holding the cities
cities = []

# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count
len(cities)

746

In [4]:
# Perform an API call with the OpenWeatherMap

# Build URL
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Create an empty list to hold the weather data
city_data = []

#Print the beginning of the logging
print("Beginning Data Retrieval")
print("------------------------")

# Create counters 
record_count = 1
set_count = 1

# Loop through all the cities in the list
for i, city in enumerate(cities):
        
        # Group cities in sets of 50 for logging purposes
        if (i % 50 == 0 and i >= 50):
            set_count += 1
            record_count = 1
        # Create endpoint URL with each city
        city_url = url + "&q=" + city.replace(" ","+")
        
        # Log the URL, record, and set numbers and the city
        #print(f"Processing Record {record_count} of Set {set_count} | {city}")

        # Add 1 to the record count
        record_count += 1


        # Run an API request for each of the cities.
        try:
            # Parse the JSON and retrieve data.
            city_weather = requests.get(city_url).json()
            # Parse out the needed data.
            city_country = city_weather["sys"]["country"]
            city_lat = city_weather["coord"]["lat"]
            city_lng = city_weather["coord"]["lon"]
            city_max_temp = city_weather["main"]["temp_max"]
            city_humidity = city_weather["main"]["humidity"]
            city_clouds = city_weather["clouds"]["all"]
            city_wind = city_weather["wind"]["speed"]
            city_desc = city_weather["weather"][0]["description"]

            # Append the city information into city_data list.
            city_data.append({"City": city.title(),
                              "Country": city_country,
                              "Lat": city_lat,
                              "Lng": city_lng,
                              "Max Temp": city_max_temp,
                              "Humidity": city_humidity,
                              "Cloudiness": city_clouds,
                              "Wind Speed": city_wind,
                              "Current Description": city_desc,
                              })

        # If an error is experienced, skip the city.
        except:
            print("City not found. Skipping...")
            pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval
------------------------
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping..

In [6]:
# Add the data to a new DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,53.58,83,95,4.29,overcast clouds
1,Severo-Kurilsk,RU,50.6789,156.1250,41.56,79,86,13.62,overcast clouds
2,Rikitea,PF,-23.1203,-134.9692,75.13,85,84,21.00,light rain
3,Mataura,NZ,-46.1927,168.8643,48.25,97,100,14.07,light rain
4,Seymchan,RU,62.8833,152.4333,5.76,83,100,7.16,overcast clouds
5,Zhigansk,RU,66.7697,123.3711,2.07,92,100,21.72,overcast clouds
6,Garowe,SO,8.4054,48.4845,71.60,84,40,8.88,scattered clouds
7,Cabo San Lucas,MX,22.8909,-109.9124,88.52,71,20,11.99,few clouds
8,Ushuaia,AR,-54.8000,-68.3000,51.46,46,0,18.41,clear sky
9,Heishan,CN,41.6892,122.1128,38.86,81,100,14.81,overcast clouds


In [8]:
# Export the DataFrame as a CSV file and save it as WeatherPy_Database.csv in the Weather_Datavase folder
# Create the output file (CSV)
output_data_file = "../Weather_Database/WeatherPy_Database.csv"

# Export the City_data into a CSV
city_data_df.to_csv(output_data_file, index_label="City_ID")